In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy import linalg
from Mesh import Mesh

import matplotlib.pyplot as plt


In [2]:
# Convert to torch tensors (double precision for better numerical stability)
torch.set_default_dtype(torch.double)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
m = Mesh('data/coil_1.2_MM.obj')

centroid = m.verts.mean(0)
std_max = m.verts.std(0).max()

verts_new = (m.verts - centroid)/std_max

m = Mesh(verts = verts_new, connectivity = m.connectivity)

print('Computing Laplacian')
K, M = m.computeLaplacian()

# following Finite Elements methodology 
# K is stiffness matrix, M is mass matrix
# The problem to solve becomes 
# K*u = lambda * M*u
print('Computing eigen values')
eigvals, eigvecs = linalg.eigh(K,M)

Computing Laplacian
Computing eigen values


In [4]:
# send all relevant numpy arrays to torch tensors
K = torch.from_numpy(K).to(device)
M = torch.from_numpy(M).to(device)
X = torch.from_numpy(m.verts).to(device)

In [5]:
# in the paper we used 50 eigenvalues so set k to 50
k = 50

In [9]:
class MLP(nn.Module):
    """
    Multilayer Perceptron for mapping coordinates to k eigenmodes.
    Uses SiLU (Swish) activation for better gradient flow than Tanh.
    """
    def __init__(self, in_dim=3, out_dim=k, hidden=[64, 64]):
        super().__init__()
        layers = []
        last = in_dim
        for h in hidden:
            # Using nn.SiLU (Swish) instead of nn.Tanh
            layers.append(nn.Linear(last, h))
            layers.append(nn.SiLU())
            last = h
        layers.append(nn.Linear(last, out_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)  # returns (N, k)

# --- 3. Model Instantiation and Initialization ---

# Instantiate model
model = MLP().to(device)

# Initialize all layers (Xavier), final layer small (Best practice for PINNs)
for name, p in model.named_parameters():
    if 'net' in name:
        # Standard Xavier for hidden layers (weights)
        if p.dim() > 1 and name.split('.')[1] != str(len(model.net) - 1):
            nn.init.xavier_uniform_(p.data)
        # Final Linear layer: Small weights and zero bias
        if name.split('.')[1] == str(len(model.net) - 1):
            if p.ndim == 2:
                # Weights: Very small normal distribution
                nn.init.normal_(p.data, std=1e-3)
            else:
                # Biases: Zero
                nn.init.zeros_(p.data)

# --- 4. Training Setup ---

# Hyperparameters
lambda_orth = 1.0           # CRITICAL: Weight for the orthogonality loss.
                            # Adjust this if orth_loss and eig_loss have vastly different scales.
lr_start = 0.01
lr_end = 0.0001
max_epochs = 10_000         # Reduced epochs for quicker demonstration
print_every = 500
loss_history = []

optimizer = optim.Adam(model.parameters(), lr=lr_start)
decay_factor = (lr_end / lr_start) ** (1 / max_epochs)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay_factor)

# --- 5. Training Loop ---

print("\n--- Starting Training ---")
identity_k = torch.eye(k, device=device)

for epoch in range(1, max_epochs + 1):
    model.train()
    optimizer.zero_grad()
    
    # Forward Pass
    U = model(X)  # N x k (Basis functions evaluated at coordinates X)

    # Calculate Loss Components
    
    # 1. Eigenvalue Loss (Minimize Rayleigh Quotient)
    # The term U.T @ (K @ U) results in a k x k matrix. The trace sums the k eigenvalues.
    eig_loss = torch.trace(U.T @ (K @ U)) 
    
    # 2. Orthogonality Loss (Ensure M-orthonormality: U^T M U = I)
    B = U.T @ (M @ U)        # k x k (Orthogonality matrix)
    orth_loss = torch.norm(B - identity_k, p='fro')**2

    # Total Loss (Weighted sum)
    # Improvement: Explicitly use lambda_orth for weighting
    loss = eig_loss + lambda_orth * orth_loss

    # Backpropagation and Step
    loss.backward()
    optimizer.step()
    scheduler.step()
    
    # Logging and Analysis
    loss_history.append(loss.item())

    if epoch % print_every == 0 or epoch == 1:
        # Calculate the approximate eigenvalues by extracting the diagonal of U.T @ K @ U
        # Note: These values should be minimized, and they represent the k smallest modes.
        approx_vals = torch.diag(U.T @ (K @ U)).detach().cpu().numpy()
        
        # Sort and print the approximate eigenvalues for meaningful comparison
        sorted_vals = np.sort(approx_vals)
        
        current_lr = scheduler.get_last_lr()[0]
        
        print(
            f"Epoch {epoch:<5}, LR={current_lr:.6f}, "
            f"Total Loss={loss.item():.4f}, "
            f"Eig Loss={eig_loss.item():.4f}, "
            f"Orth Loss={orth_loss.item():.4f}"
        )
        # print(f"  Approx Eigenvalues (Min k): {sorted_vals}")

print("--- Training Complete ---")


--- Starting Training ---
Epoch 1    , LR=0.009995, Total Loss=49.9957, Eig Loss=0.0004, Orth Loss=49.9954
Epoch 500  , LR=0.007943, Total Loss=47.0539, Eig Loss=1.5662, Orth Loss=45.4877
Epoch 1000 , LR=0.006310, Total Loss=46.5962, Eig Loss=2.0098, Orth Loss=44.5864
Epoch 1500 , LR=0.005012, Total Loss=46.1922, Eig Loss=2.7743, Orth Loss=43.4179
Epoch 2000 , LR=0.003981, Total Loss=46.0724, Eig Loss=2.8329, Orth Loss=43.2396
Epoch 2500 , LR=0.003162, Total Loss=46.0244, Eig Loss=2.8508, Orth Loss=43.1736
Epoch 3000 , LR=0.002512, Total Loss=46.0040, Eig Loss=2.8743, Orth Loss=43.1297
Epoch 3500 , LR=0.001995, Total Loss=45.9602, Eig Loss=3.1526, Orth Loss=42.8076
Epoch 4000 , LR=0.001585, Total Loss=45.9368, Eig Loss=3.2058, Orth Loss=42.7310
Epoch 4500 , LR=0.001259, Total Loss=45.9196, Eig Loss=3.3049, Orth Loss=42.6148
Epoch 5000 , LR=0.001000, Total Loss=45.9053, Eig Loss=3.3778, Orth Loss=42.5275
Epoch 5500 , LR=0.000794, Total Loss=45.8934, Eig Loss=3.4101, Orth Loss=42.4833
E

In [ ]:
np.set_printoptions(suppress=True, precision=6)


# Final Eigenvalue Check
model.eval()
with torch.no_grad():
    U_final = model(X)
    final_rayleigh_matrix = U_final.T @ (K @ U_final)
    final_ortho_matrix = U_final.T @ (M @ U_final)

    final_eigenvalues = torch.diag(final_rayleigh_matrix).cpu().numpy()
    final_eigenvalues.sort()

    print("\n--- Final Results ---")
    print(f"Final Approximate Eigenvalues (Sorted): {np.round(final_eigenvalues[:5], 6)}")
    print("Reference eigenvalues (first k):   ", np.round(eigvals[:5], 6))
    
    # Print the Orthogonality Matrix (should be close to Identity)
    print("\nFinal Orthogonality Matrix (U^T M U):")
    print(final_ortho_matrix.cpu().numpy().round(4))

In [ ]:
# Plot the loss history
plt.figure(figsize=(10, 5))
plt.plot(loss_history, label='Total Loss')
plt.title('Training Loss History')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.yscale('log')
plt.grid(True, which="both", ls="--")
plt.legend()
plt.show()